In [404]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import confusion_matrix
 


In [405]:
df = pd.read_csv('Spam-k.csv',encoding='latin')

In [406]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [407]:
df.drop({'Unnamed: 2','Unnamed: 3', 'Unnamed: 4'}, axis=1, inplace=True)

In [408]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [409]:
df.rename(columns = {'v1':'Type' , 'v2':'Msg'}, inplace = True) 

In [410]:
df.head()

,Type,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [411]:
df.shape

(5572, 2)

In [412]:


##data Processing, cleaning, structuring,
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
from nltk.stem import WordNetLemmatizer
import re

def filtering(text):
    lemmatizer= WordNetLemmatizer()
    for index,row in df.iterrows():
            filter_sentence=[]
            sentence= row['Msg']
            sentence= re.sub(r'[^\w\s]','',sentence)
            words= nltk.word_tokenize(sentence)
            words= [w for w in words if not w in stop_words]
            for word in words:
                filter_sentence.append(lemmatizer.lemmatize(word))
            print(filter_sentence)
            df.loc[index, 'Msg']= filter_sentence

In [413]:
df['Msg']=df.apply(lambda x: x['Msg'],axis=1)

In [414]:
df.head()

,Type,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [415]:
# train test split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Msg'], df['Type'], test_size = 0.25, random_state = 1)

In [416]:

# vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [417]:
# training the classifier 
svm = svm.SVC(C=10000)
svm.fit(X_train, y_train)

    

SVC(C=10000)

In [418]:
# testing testing set 
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test) 
print(confusion_matrix(y_test, y_pred))

# test against new messages 
def pred(msg):
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]


[[1212    1]
 [  15  165]]


In [419]:
print(accuracy_score(y_test, y_pred))
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

0.9885139985642498


predicted     
                  ham spam
actual ham       1212    1
       spam        15  165